<a href="https://colab.research.google.com/github/rajnishdascse/movie-recommendation-app/blob/main/preprocessing_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2022 movies from Wikipedia

> Indented block



In [46]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [47]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [48]:
tables = soup.find_all('table',class_='wikitable sortable')

In [49]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [50]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [51]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[5],NaN
4,JANUARY,14.0,Hotel Transylvania: Transformania,Amazon Studios / Columbia Pictures / Sony Pict...,"Jennifer Kluska, Derek Drymon (directors); Amo...",[6],NaN
...,...,...,...,...,...,...,...
177,DECEMBER,21.0,Puss in Boots: The Last Wish,Universal Pictures / DreamWorks Animation,"Joel Crawford (director); Antonio Banderas, Sa...",NaN,[152]
178,DECEMBER,21.0,I Wanna Dance with Somebody,TriStar Pictures / Black Label Media,Kasi Lemmons (director); Anthony McCarten (scr...,NaN,[52]
179,DECEMBER,25.0,Babylon,Paramount Pictures,Damien Chazelle (director/screenplay); Brad Pi...,NaN,[153]
180,DECEMBER,25.0,A Man Called Otto,Columbia Pictures,"Marc Forster (director), David Magee (screenpl...",NaN,[52]


In [53]:
df_2022 = df[['Title','Cast and crew']]

In [54]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo..."
...,...,...
177,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa..."
178,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...
179,Babylon,Damien Chazelle (director/screenplay); Brad Pi...
180,A Man Called Otto,"Marc Forster (director), David Magee (screenpl..."


In [55]:
!pip install tmdbv3api

In [56]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '225ac8079ea25738ddc6a4c08757dd89'

In [57]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [59]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
df_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure
...,...,...,...
177,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa...",Animation Adventure Comedy Family Fantasy
178,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama
179,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller
180,A Man Called Otto,"Marc Forster (director), David Magee (screenpl...",Drama Comedy


In [61]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [62]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [64]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [66]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [67]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [68]:

df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [69]:
df_2022

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn
...,...,...,...,...,...,...,...
177,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa...",Animation Adventure Comedy Family Fantasy,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén
178,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci
179,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire
180,A Man Called Otto,"Marc Forster (director), David Magee (screenpl...",Drama Comedy,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller


In [70]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [71]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [72]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,Hotel Transylvania: Transformania
...,...,...,...,...,...,...
177,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,Puss in Boots: The Last Wish
178,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,I Wanna Dance with Somebody
179,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire,Action Adventure Science Fiction Thriller,Babylon
180,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Drama Comedy,A Man Called Otto


In [73]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']

In [74]:
new_df22 = new_df22.dropna(how='any')

In [75]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
5,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Thriller,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
176,David F. Sandberg,Zachary Levi,Jack Dylan Grazer,Rachel Zegler,Comedy Action Fantasy,shazam! fury of the gods,Zachary Levi Jack Dylan Grazer Rachel Zegler D...
177,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,puss in boots: the last wish,Joel Crawford (director); Antonio Banderas Sal...
178,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,i wanna dance with somebody,Naomi Ackie Ashton Sanders Stanley Tucci Kasi ...
179,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire,Action Adventure Science Fiction Thriller,babylon,Brad Pitt Margot Robbie Tobey Maguire Damien C...


In [77]:
old_df = pd.read_csv('main_updated_data.csv')

In [78]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Animation Comedy Family,toy story,Tom Hanks Tim Allen Don Rickles John Lasseter ...
1,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family,jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Joe...
2,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy,grumpier old men,Walter Matthau Jack Lemmon Ann-Margret Howard ...
3,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Comedy Drama Romance,waiting to exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Comedy,father of the bride part ii,Steve Martin Diane Keaton Martin Short Charles...
...,...,...,...,...,...,...,...
37314,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
37315,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
37316,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
37317,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [79]:
final_df = old_df.append(new_df22,ignore_index=True)

In [80]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Animation Comedy Family,toy story,Tom Hanks Tim Allen Don Rickles John Lasseter ...
1,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family,jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Joe...
2,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy,grumpier old men,Walter Matthau Jack Lemmon Ann-Margret Howard ...
3,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Comedy Drama Romance,waiting to exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Comedy,father of the bride part ii,Steve Martin Diane Keaton Martin Short Charles...
...,...,...,...,...,...,...,...
37482,David F. Sandberg,Zachary Levi,Jack Dylan Grazer,Rachel Zegler,Comedy Action Fantasy,shazam! fury of the gods,Zachary Levi Jack Dylan Grazer Rachel Zegler D...
37483,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,puss in boots: the last wish,Joel Crawford (director); Antonio Banderas Sal...
37484,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama,i wanna dance with somebody,Naomi Ackie Ashton Sanders Stanley Tucci Kasi ...
37485,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire,Action Adventure Science Fiction Thriller,babylon,Brad Pitt Margot Robbie Tobey Maguire Damien C...


In [81]:
final_df.to_csv('main_final_data.csv',index=False)